In [99]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import streamlit as st
import os
import warnings
warnings.filterwarnings("ignore")

In [100]:
# Loading data
test_data = pd.read_csv('C:/Users/Hp/Desktop/Capstone Project/Data/test_features.csv')
train_data = pd.read_csv('C:/Users/Hp/Desktop/Capstone Project/Data/train_features.csv')
train_labels = pd.read_csv('C:/Users/Hp/Desktop/Capstone Project/Data/train_labels.csv')

In [101]:
datasets = [("Test Data", test_data), ("Train Data", train_data), ("Train Labels", train_labels)]
for name, data in datas:
    print(f"\n{name} Summary:")
    print(data.info())
    print("\nStatistical Summary:")
    print(data.describe())
    print("*" * 100)



Test Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819 entries, 0 to 818
Columns: 184 entries, uid to j11_12
dtypes: float64(140), object(44)
memory usage: 1.1+ MB
None

Statistical Summary:
         n_mar_03  migration_03  adl_dress_03  adl_walk_03  adl_bath_03  \
count  568.000000    570.000000    542.000000   569.000000   569.000000   
mean     1.165493      0.077193      0.047970     0.012302     0.012302   
std      0.535254      0.267132      0.213901     0.110328     0.110328   
min      0.000000      0.000000      0.000000     0.000000     0.000000   
25%      1.000000      0.000000      0.000000     0.000000     0.000000   
50%      1.000000      0.000000      0.000000     0.000000     0.000000   
75%      1.000000      0.000000      0.000000     0.000000     0.000000   
max      4.000000      1.000000      1.000000     1.000000     1.000000   

       adl_eat_03  adl_bed_03  adl_toilet_03    n_adl_03  iadl_money_03  ...  \
count  569.000000  569.000000     

In [102]:
# Checking dataset dimensions and data types
for name, data in datasets:
    print(f"\n--- {name} Overview ---")
    print("Shape:", data.shape)
    print("Data Types:")
    print(data.dtypes)
    print("Null Values:")
    print(data.isnull().sum())
    print("Unique Values:")
    print(data.nunique())
    print('*********************************************************************')


--- Test Data Overview ---
Shape: (819, 184)
Data Types:
uid            object
age_03         object
urban_03       object
married_03     object
n_mar_03      float64
               ...   
a21_12        float64
a22_12         object
a33b_12        object
a34_12         object
j11_12         object
Length: 184, dtype: object
Null Values:
uid             0
age_03        249
urban_03      249
married_03    249
n_mar_03      251
             ... 
a21_12        808
a22_12        809
a33b_12       808
a34_12        277
j11_12         24
Length: 184, dtype: int64
Unique Values:
uid           819
age_03          5
urban_03        2
married_03      4
n_mar_03        5
             ... 
a21_12          5
a22_12          4
a33b_12         2
a34_12          2
j11_12          3
Length: 184, dtype: int64
*********************************************************************

--- Train Data Overview ---
Shape: (3276, 184)
Data Types:
uid            object
age_03         object
urban_03       object
m

In [103]:
# Count and percentage of missing values in each column
missing_values_count = train_data.isnull().sum()
missing_percentage = (missing_values_count / len(train_data)) * 100

In [104]:
missing_values_count


uid              0
age_03        1036
urban_03      1034
married_03    1034
n_mar_03      1054
              ... 
a21_12        3234
a22_12        3240
a33b_12       3234
a34_12        1164
j11_12          75
Length: 184, dtype: int64

In [105]:
missing_percentage

uid            0.000000
age_03        31.623932
urban_03      31.562882
married_03    31.562882
n_mar_03      32.173382
                ...    
a21_12        98.717949
a22_12        98.901099
a33b_12       98.717949
a34_12        35.531136
j11_12         2.289377
Length: 184, dtype: float64

In [106]:
# Display missing values and their percentages
missing_data = pd.DataFrame({
    'Missing Values': missing_values_count,
    'Percentage': missing_percentage
})
print(missing_data[missing_data['Missing Values'] > 0])  

            Missing Values  Percentage
age_03                1036   31.623932
urban_03              1034   31.562882
married_03            1034   31.562882
n_mar_03              1054   32.173382
edu_gru_03            1044   31.868132
...                    ...         ...
a21_12                3234   98.717949
a22_12                3240   98.901099
a33b_12               3234   98.717949
a34_12                1164   35.531136
j11_12                  75    2.289377

[182 rows x 2 columns]


In [107]:
# Identifying columns to drop (those with more than 40% missing values)
columns_to_drop = missing_data[missing_data['Percentage'] > 40].index

In [108]:
train_data_cleaned = train_data.drop(columns=columns_to_drop)

In [109]:
# Displaying the cleaned DataFrame and the columns dropped
print("Columns Dropped:")
print(columns_to_drop.tolist())
print("\nCleaned DataFrame:")
print(train_data_cleaned.head())

Columns Dropped:
['bmi_03', 'decis_famil_03', 'sgender_03', 'rjob_hrswk_03', 'rjlocc_m_03', 'rjob_end_03', 'rjobend_reason_03', 'searnings_03', 'sinc_pension_03', 'rjob_hrswk_12', 'rjlocc_m_12', 'rjob_end_12', 'rjobend_reason_12', 'a16a_12', 'a21_12', 'a22_12', 'a33b_12']

Cleaned DataFrame:
    uid    age_03     urban_03  married_03  n_mar_03    edu_gru_03  \
0  aace       NaN          NaN         NaN       NaN           NaN   
1  aanz       NaN          NaN         NaN       NaN           NaN   
2  aape       NaN          NaN         NaN       NaN           NaN   
3  aard  1. 50–59  1. 100,000+  3. Widowed       1.0  3. 7–9 years   
4  ablr       NaN          NaN         NaN       NaN           NaN   

  n_living_child_03  migration_03 glob_hlth_03  adl_dress_03  ...  \
0               NaN           NaN          NaN           NaN  ...   
1               NaN           NaN          NaN           NaN  ...   
2               NaN           NaN          NaN           NaN  ...   
3         

## Features grouping

In [110]:
# Print all column names as a list
print(list(train_data_cleaned.columns))



['uid', 'age_03', 'urban_03', 'married_03', 'n_mar_03', 'edu_gru_03', 'n_living_child_03', 'migration_03', 'glob_hlth_03', 'adl_dress_03', 'adl_walk_03', 'adl_bath_03', 'adl_eat_03', 'adl_bed_03', 'adl_toilet_03', 'n_adl_03', 'iadl_money_03', 'iadl_meds_03', 'iadl_shop_03', 'iadl_meals_03', 'n_iadl_03', 'depressed_03', 'hard_03', 'restless_03', 'happy_03', 'lonely_03', 'enjoy_03', 'sad_03', 'tired_03', 'energetic_03', 'n_depr_03', 'cesd_depressed_03', 'hypertension_03', 'diabetes_03', 'resp_ill_03', 'arthritis_03', 'hrt_attack_03', 'stroke_03', 'cancer_03', 'n_illnesses_03', 'exer_3xwk_03', 'alcohol_03', 'tobacco_03', 'test_chol_03', 'test_tuber_03', 'test_diab_03', 'test_pres_03', 'hosp_03', 'visit_med_03', 'out_proc_03', 'visit_dental_03', 'imss_03', 'issste_03', 'pem_def_mar_03', 'insur_private_03', 'insur_other_03', 'insured_03', 'decis_personal_03', 'employment_03', 'age_12', 'urban_12', 'married_12', 'n_mar_12', 'edu_gru_12', 'n_living_child_12', 'migration_12', 'glob_hlth_12', '

In [111]:
# Grouping the features based on their types
numerical_features = [
    'age_03', 'age_12', 'rearnings_03', 'hincome_03', 'hinc_business_03', 
    'hinc_rent_03', 'hinc_assets_03', 'hinc_cap_03', 'rinc_pension_03',
    'rearnings_12', 'searnings_12', 'hincome_12', 'hinc_business_12', 
    'hinc_rent_12', 'hinc_assets_12', 'hinc_cap_12', 'rinc_pension_12', 
    'sinc_pension_12', 'n_mar_03', 'n_living_child_03', 'n_mar_12', 
    'n_living_child_12', 'glob_hlth_03', 'glob_hlth_12', 'bmi_12',
    'visit_med_03', 'visit_med_12', 'test_chol_03', 'test_chol_12',
    'test_tuber_03', 'test_tuber_12', 'test_diab_03', 'test_diab_12', 
    'test_pres_03', 'test_pres_12'
]

ordinal_features = [
    'glob_hlth_03', 'adl_dress_03', 'adl_walk_03', 'adl_bath_03', 
    'adl_eat_03', 'adl_bed_03', 'adl_toilet_03', 'n_adl_03', 'iadl_money_03', 
    'iadl_meds_03', 'iadl_shop_03', 'iadl_meals_03', 'n_iadl_03', 
    'n_depr_03', 'depressed_03', 'happy_03', 'lonely_03', 'enjoy_03', 
    'sad_03', 'tired_03', 'energetic_03', 'cesd_depressed_03',
    'glob_hlth_12', 'adl_dress_12', 'adl_walk_12', 'adl_bath_12', 
    'adl_eat_12', 'adl_bed_12', 'adl_toilet_12', 'n_adl_12', 'iadl_money_12', 
    'iadl_meds_12', 'iadl_shop_12', 'iadl_meals_12', 'n_iadl_12', 
    'n_depr_12', 'depressed_12', 'happy_12', 'lonely_12', 'enjoy_12', 
    'sad_12', 'tired_12', 'energetic_12', 'cesd_depressed_12'
]

nominal_features = [
    'hypertension_03', 'diabetes_03', 'arthritis_03', 'hrt_attack_03', 
    'stroke_03', 'cancer_03', 'exer_3xwk_03', 'alcohol_03', 'tobacco_03', 
    'hosp_03', 'imss_03', 'issste_03', 'insur_private_03', 'insur_other_03', 
    'insured_03', 'employment_03', 'hypertension_12', 'diabetes_12', 
    'arthritis_12', 'hrt_attack_12', 'stroke_12', 'cancer_12', 'exer_3xwk_12', 
    'alcohol_12', 'tobacco_12', 'hosp_12', 'imss_12', 'issste_12', 
    'insur_private_12', 'insur_other_12', 'insured_12', 'employment_12', 
    'volunteer_12', 'attends_class_12', 'attends_club_12', 'reads_12', 
    'games_12', 'tv_12'
]

demographic_features = [
    'ragender', 'sgender_12', 'rameduc_m', 'rafeduc_m', 'rrelgimp_03', 'rrelgimp_12'
]

In [112]:
# Function to handle missing values based on feature groups
def handle_missing_values(train_data_cleaned):
    # Numerical Features: Mean/Median Imputation (only for numeric columns)
    for col in numerical_features:
        if pd.api.types.is_numeric_dtype(train_data_cleaned[col]):
            train_data_cleaned[col].fillna(train_data_cleaned[col].mean(), inplace=True)
    
    # Ordinal Features: Mode Imputation
    for col in ordinal_features:
        train_data_cleaned[col].fillna(train_data_cleaned[col].mode()[0], inplace=True)
    
    # Nominal Features: Mode Imputation
    for col in nominal_features:
        train_data_cleaned[col].fillna(train_data_cleaned[col].mode()[0], inplace=True)
    
    # Demographic Features: Mode Imputation
    for col in demographic_features:
        train_data_cleaned[col].fillna(train_data_cleaned[col].mode()[0], inplace=True)
    
    return train_data_cleaned

# Apply the function to clean the dataframe
train_data_cleaned = handle_missing_values(train_data_cleaned)



In [113]:
train_data_cleaned.head()

,uid,age_03,urban_03,married_03,n_mar_03,edu_gru_03,n_living_child_03,migration_03,glob_hlth_03,adl_dress_03,...,hinc_assets_12,hinc_cap_12,rinc_pension_12,sinc_pension_12,rrelgimp_12,rrfcntx_m_12,rsocact_m_12,rrelgwk_12,a34_12,j11_12
0,aace,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4. Fair,0.0,...,0.0,10000.0,0.0,0.00000,2.somewhat important,9.Never,9.Never,0.No,NaN,Concrete 2
1,aanz,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4. Fair,0.0,...,0.0,0.0,0.0,0.00000,1.very important,9.Never,1.Almost every day,0.No,NaN,Concrete 2
2,aape,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4. Fair,0.0,...,0.0,0.0,0.0,0.00000,2.somewhat important,6.2 or 3 times a month,2.4 or more times a week,0.No,NaN,"Wood, mosaic, or other covering 1"
3,aard,1. 50–59,"1. 100,000+",3. Widowed,1.000000,3. 7–9 years,1. 1 or 2,0.0,4. Fair,0.0,...,0.0,0.0,0.0,11936.87231,1.very important,4.Once a week,9.Never,1.Yes,No 2,Concrete 2
4,ablr,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4. Fair,0.0,...,0.0,0.0,0.0,0.00000,1.very important,3.2 or 3 times a week,3.2 or 3 times a week,0.No,NaN,"Wood, mosaic, or other covering 1"


In [114]:
# Function to separate numeric part from string-encoded categorical values
def clean_encoded_strings(train_data_cleaned, columns):
    for col in columns:
        if col in train_data_cleaned.columns:  # Check if column exists
            # Extract numeric part only (if it exists)
            train_data_cleaned[col] = train_data_cleaned[col].astype(str).fillna('').str.extract('(\d+)').astype(float)
    return train_data_cleaned

# List of columns that need cleaning for encoded values
encoded_columns = [
    'married_03', 'edu_gru_03', 'n_living_child_03', 'glob_hlth_03', 'bmi_03', 'decis_famil_03', 
    'employment_03', 'age_12', 'urban_12', 'married_12', 'edu_gru_12', 'n_living_child_12', 
    'glob_hlth_12', 'bmi_12', 'decis_famil_12', 'decis_personal_12', 'employment_12', 
    'satis_ideal_12', 'satis_excel_12', 'satis_fine_12', 'cosas_imp_12', 'wouldnt_change_12', 
    'memory_12', 'ragender', 'rameduc_m', 'rafeduc_m', 'sgender_03', 'rjlocc_m_03', 
    'rjobend_reason_03', 'rrelgimp_03', 'sgender_12', 'rjlocc_m_12', 'rjobend_reason_12', 
    'rrelgimp_12', 'rrfcntx_m_12', 'rsocact_m_12', 'rrelgwk_12', 'a34_12', 'j11_12'
]

# Clean the encoded columns
train_data_cleaned = clean_encoded_strings(train_data_cleaned, encoded_columns)



In [115]:
train_data_cleaned.shape

(3276, 167)

In [116]:
train_data_cleaned.head()

,uid,age_03,urban_03,married_03,n_mar_03,edu_gru_03,n_living_child_03,migration_03,glob_hlth_03,adl_dress_03,...,hinc_assets_12,hinc_cap_12,rinc_pension_12,sinc_pension_12,rrelgimp_12,rrfcntx_m_12,rsocact_m_12,rrelgwk_12,a34_12,j11_12
0,aace,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4.0,0.0,...,0.0,10000.0,0.0,0.00000,2.0,9.0,9.0,0.0,NaN,2.0
1,aanz,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4.0,0.0,...,0.0,0.0,0.0,0.00000,1.0,9.0,1.0,0.0,NaN,2.0
2,aape,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4.0,0.0,...,0.0,0.0,0.0,0.00000,2.0,6.0,2.0,0.0,NaN,1.0
3,aard,1. 50–59,"1. 100,000+",3.0,1.000000,3.0,1.0,0.0,4.0,0.0,...,0.0,0.0,0.0,11936.87231,1.0,4.0,9.0,1.0,2.0,2.0
4,ablr,NaN,NaN,NaN,1.134113,NaN,NaN,NaN,4.0,0.0,...,0.0,0.0,0.0,0.00000,1.0,3.0,3.0,0.0,NaN,1.0


In [126]:


missing_values = train_data_cleaned.isnull().sum()


missing_values_df = pd.DataFrame(missing_values, columns=['Missing Count']).reset_index()
missing_values_df.columns = ['Column Name', 'Missing Count']


non_zero_missing_df = missing_values_df[missing_values_df['Missing Count'] > 0]


non_zero_missing_df = non_zero_missing_df.sort_values(by='Missing Count', ascending=False)


print(non_zero_missing_df)


           Column Name  Missing Count
116     decis_famil_12           1226
57   decis_personal_03           1178
23         restless_03           1172
22             hard_03           1172
165             a34_12           1164
6    n_living_child_03           1046
5           edu_gru_03           1044
1               age_03           1036
7         migration_03           1035
39      n_illnesses_03           1035
34         resp_ill_03           1034
3           married_03           1034
2             urban_03           1034
53      pem_def_mar_03           1034
50     visit_dental_03           1034
49         out_proc_03           1034
98              bmi_12            438
120        vax_pneu_12            291
134     satis_ideal_12            277
138  wouldnt_change_12            248
139          memory_12            245
135     satis_excel_12            239
137       cosas_imp_12            228
162       rrfcntx_m_12            225
117  decis_personal_12            225
119         

In [130]:

# 1. Categorical Columns: Fill missing values with the mode (most frequent value)
# Explanation: For categorical data, the most frequent value often makes a reasonable assumption for the missing value.
categorical_columns = [
    'age_03', 'edu_gru_03', 'married_03', 'urban_03'
]

for col in categorical_columns:
    # Check if mode exists and handle the case where mode is empty
    mode_value = train_data_cleaned[col].mode()
    if not mode_value.empty:
        train_data_cleaned[col].fillna(mode_value[0], inplace=True)
    else:
        print(f"Warning: Column {col} has no mode (empty column).")

# 2. Binary Columns: Fill missing values with the most frequent value in the column
# Explanation: Since binary columns typically have two values, using the most common (e.g., 0 or 1) aligns with the prevailing trend.
binary_columns = [
    'migration_03', 'pem_def_mar_03', 'resp_ill_03', 'hard_03'
]

for col in binary_columns:
    most_frequent = train_data_cleaned[col].value_counts().idxmax() if not train_data_cleaned[col].value_counts().empty else None
    if most_frequent is not None:
        train_data_cleaned[col].fillna(most_frequent, inplace=True)
    else:
        print(f"Warning: Column {col} has no values to determine most frequent.")

# 3. Numerical Columns: Fill missing values with the median
# Explanation: For numerical columns, the median is a robust measure, especially if the data is skewed.
numerical_columns = [
    'n_living_child_03', 'n_illnesses_03', 'vax_pneu_12'
]

for col in numerical_columns:
    if train_data_cleaned[col].notna().sum() > 0:
        train_data_cleaned[col].fillna(train_data_cleaned[col].median(), inplace=True)
    else:
        print(f"Warning: Column {col} has no numeric values for median calculation.")

# 4. Year-Based Feature Engineering: Interpolate missing values between 2003 and 2012 for paired columns
# Explanation: For columns where data is available across multiple years, filling missing values in one year
# by referencing the other year’s value can provide a reasonable estimate, assuming stability over time.
paired_columns = [
    ('resp_ill_03', 'resp_ill_12')
]

for col_03, col_12 in paired_columns:
    train_data_cleaned[col_03].fillna(train_data_cleaned[col_12], inplace=True)
    train_data_cleaned[col_12].fillna(train_data_cleaned[col_03], inplace=True)

# 5. New Feature Creation for Missing Indicators: Add binary columns to indicate missing data
# Explanation: Adding binary "missing" indicators helps capture missingness as a feature, which can
# contribute useful signals during analysis or modeling.
missing_indicators = [
    'depressed_03', 'decis_famil_12'
]

for col in missing_indicators:
    train_data_cleaned[f'{col}_missing'] = train_data_cleaned[col].isnull().astype(int)


In [131]:
train_data_cleaned.isnull().sum()

uid                          0
age_03                       0
urban_03                     0
married_03                   0
n_mar_03                     0
                          ... 
rrelgwk_12                 218
a34_12                    1164
j11_12                      75
depressed_03_missing         0
decis_famil_12_missing       0
Length: 169, dtype: int64

In [133]:
missing_values_after = train_data_cleaned.isnull().sum()


missing_values_after_df = pd.DataFrame(missing_values_after, columns=['Missing Count']).reset_index()
missing_values_after_df.columns = ['Column Name', 'Missing Count']


non_zero_missing_after_df = missing_values_after_df[missing_values_after_df['Missing Count'] > 0]


non_zero_missing_after_df = non_zero_missing_after_df.sort_values(by='Missing Count', ascending=False)

print(non_zero_missing_after_df)


           Column Name  Missing Count
116     decis_famil_12           1226
57   decis_personal_03           1178
23         restless_03           1172
165             a34_12           1164
50     visit_dental_03           1034
49         out_proc_03           1034
98              bmi_12            438
134     satis_ideal_12            277
138  wouldnt_change_12            248
139          memory_12            245
135     satis_excel_12            239
137       cosas_imp_12            228
162       rrfcntx_m_12            225
117  decis_personal_12            225
136      satis_fine_12            223
119         vax_flu_12            223
81         restless_12            219
129     table_games_12            219
164         rrelgwk_12            218
80             hard_12            218
130  comms_tel_comp_12            218
123      care_child_12            217
122      care_adult_12            216
131        act_mant_12            216
133          sewing_12            216
163       rs

In [118]:
duplicate_rows = data.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)


Number of duplicate rows: 0
